In [1]:
import matplotlib.pyplot as plt
import numpy as np
import argparse 
import glob
import cv2 
import os

from collections import defaultdict
%matplotlib inline

In [2]:
def read_image_files(image_path, conversion=cv2.COLOR_BGR2RGB):
    image_files = glob.glob(os.path.join(image_path, '*')) #function in order to convert the images pass through the image path and conversion
    image_list = [(os.path.basename(f),cv2.imread(f,conversion)) for f in image_files]
    image_dict = {file:image for (file,image) in image_list}
    return image_dict
mylist = (0,1,2,3,4,5,6)
number_of_bins = 8
def bin_number(incriments):
    number_bins = number_of_bins * incriments
    return number_bins
    


def generate_histogram(image_dict, number_bins=8):
    histogram_dict = dict()
    for filename in image_dict: # function to take the images and create histograms for each one 
        image = image_dict[filename]
        hist0 = cv2.calcHist([image], [0], None, [number_bins], [0,256])
        hist1 = cv2.calcHist([image], [1], None, [number_bins], [0,256])
        hist2 = cv2.calcHist([image], [2], None, [number_bins], [0,256])
        overall_hist = np.concatenate([hist0,hist1,hist2]).ravel()
        hist = overall_hist / overall_hist.sum()
        histogram_dict[filename] = hist
    return histogram_dict

In [3]:

image_directory = '/home/CV/HistogramDataset/'
targets_directory = 'targets'
mystery_directory = 'mystery' # directories for all the images 
class_directory = 'class'
class_list = ['bulbasaur','charmander','pikachu','squirtle']


image_target_dict = read_image_files(image_directory + targets_directory) #call the image read
hist_target_dict = generate_histogram(image_target_dict) #create the histograms for the targets
for i in mylist:
    number_of_bins = bin_number(i)# use this to find the best bin number for the histograms
    
image_class_dict = dict()
hist_class_dict = dict()
for c in class_list:
    image_class_dict[c] = read_image_files(os.path.join(image_directory,'classes',c)) #now do the same for the class images
    hist_class_dict[c] = generate_histogram(image_class_dict[c])
    
image_mystery_dict = read_image_files(os.path.join(image_directory,'mystery')) #lastly same thing for the mystery images
hist_mystery_dict = generate_histogram(image_mystery_dict)

In [4]:
OPENCV_METHODS = (
        ("Correlation", cv2.HISTCMP_CORREL, True),
        ("Chi-Squared", cv2.HISTCMP_CHISQR, False),
        ("Intersection", cv2.HISTCMP_INTERSECT, True,), # call the histogram matching methods and whether it is in reverse or not
        ("Hellinger", cv2.HISTCMP_BHATTACHARYYA,False ))

In [5]:
def compare_histogram(image_name, target_hist_base, class_histogram_dict):
    results_dict = dict()
    for (methodName, method, reverse) in OPENCV_METHODS:
        results = {k : cv2.compareHist(target_hist_base, hist, method) for (k,hist) in sorted(class_histogram_dict.items())} #compare the histograms pass in the targets and class
        results = sorted([(v, k) for (k, v) in results.items()], reverse = reverse)
        results_dict[methodName] = results
        
    return results_dict
def compare_mystery(k,image_name, mystery_image_name, target_hist_base, target_hist_base_two):
    mystery_results_dict = dict()
    for (methodName, method, reverse) in OPENCV_METHODS:
        mystery_results = {image_name : cv2.compareHist(target_hist_base, target_hist_base_two, method)}

        mystery_results_dict[methodName] = mystery_results #create a nested dictionary containing the results of the mystery images compared with the targets
    return mystery_results_dict

In [6]:
k=0
mystery_class_compare_dict = dict()
mystery_results_dict_squirtle = dict()
mystery_results_dict_char = dict() #create dictionaries for each target pokemon to compare with the mystery pokemon
mystery_results_dict_bulb = dict()
mystery_results_dict_pikachu = dict()
for (image_name, hist_base) in hist_target_dict.items():
    target_class = image_name.split('.')[0].split('_')[1] # grab the names of each target 
    for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()): # grab the names of each mystery image
        mystery_class = image_name_mystery
        if image_name == 'target_squirtle.png': # now for each target name call the compare mystery function and compare the target to each mystery and return a nested dictionary
            mystery_results_dict_squirtle[mystery_class] = compare_mystery(k, image_name, image_name_mystery, hist_base, hist_base_two)
        elif image_name == 'target_pikachu.png':
            mystery_results_dict_pikachu[mystery_class] = compare_mystery(k, image_name, image_name_mystery, hist_base, hist_base_two)
        elif image_name == 'target_charmander.png':
            mystery_results_dict_char[mystery_class] = compare_mystery(k, image_name, image_name_mystery, hist_base, hist_base_two)
        else:
            mystery_results_dict_bulb[mystery_class] = compare_mystery(k, image_name, image_name_mystery, hist_base, hist_base_two)
        k=k+1
print(mystery_results_dict_pikachu["mystery9.png"])#["Chi-Squared"]) #now you have the test results in a dictionary

{'Correlation': {'target_pikachu.png': 0.9442435604813273}, 'Chi-Squared': {'target_pikachu.png': 1.8788743753541608}, 'Intersection': {'target_pikachu.png': 0.8180390852603523}, 'Hellinger': {'target_pikachu.png': 0.3126539246082259}}


In [7]:
not_likely_squirtle_dict=dict() #now in reading online having a negative correlation is an easy way to eliminate if it is a good match or not 
for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()):
        mystery_class = image_name_mystery
        for (img_name_mystery, score) in (mystery_results_dict_squirtle[mystery_class]['Correlation'].items()): # take the mystery results and see if the score comes back negative 
            if score<0:
                not_likely_squirtle_dict[mystery_class] = score #if it does then store it in a not likely dict in order to lesson the amount of images that need to be ran 

not_likely_pikachu_dict=dict()
for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()): #now do it for all four pokemon
        mystery_class = image_name_mystery
        for (img_name_mystery, score) in (mystery_results_dict_pikachu[mystery_class]['Correlation'].items()):
            if score<0:
                not_likely_pikachu_dict[mystery_class] = score

not_likely_char_dict=dict()
for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()):
        mystery_class = image_name_mystery
        for (img_name_mystery, score) in (mystery_results_dict_char[mystery_class]['Correlation'].items()):
            if score<0:
                not_likely_char_dict[mystery_class] = score

not_likely_bulb_dict=dict()
for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()):
        mystery_class = image_name_mystery
        for (img_name_mystery, score) in (mystery_results_dict_bulb[mystery_class]['Correlation'].items()):
            if score<0:
                not_likely_bulb_dict[mystery_class] = score


In [9]:
for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()):
    for (img_name, target_score) in sorted(mystery_results_dict_squirtle[image_name_mystery]['Chi-Squared'].items()):
        print(target_score)
        if(target_score<0):
            print('h')
            print(image_name_mystery)
        print(image_name_mystery)

19.035500856377805
mystery1.png
0.5546843375583982
mystery10.png
14.482979355922655
mystery2.png
14.590161034958513
mystery3.png
3.4285130722248445
mystery4.png
0.5792180573918561
mystery5.png
32.89657179115637
mystery6.png
5.464785033639735
mystery7.png
22.954080079992586
mystery8.png
3.830905912460245
mystery9.png


In [10]:
target_class_compare_dict = defaultdict(dict) # create the target and class compare dictionary compare the 10 class images to the target image
for (image_name, hist_base) in hist_target_dict.items():
    target_class = image_name.split('.')[0].split('_')[1]
    for class_name in class_list:
        target_class_compare_dict[target_class][class_name] = compare_histogram(image_name, hist_base, hist_class_dict[class_name]) # return a dictionary of each one

In [11]:
print (target_class_compare_dict['pikachu']['pikachu']["Correlation"])
print (target_class_compare_dict['pikachu']['pikachu']["Chi-Squared"])
print (target_class_compare_dict['pikachu']['pikachu']["Intersection"]) # call the target class compare dict and pring the results
print (target_class_compare_dict['pikachu']['pikachu']["Hellinger"])

[(0.8578661145484695, '4.png'), (0.84162808425238, '5.png'), (0.44570073193045195, '8.png'), (0.23110706517289242, '1.png'), (0.2247963657183375, '3.png'), (0.13955487527445715, '2.png'), (0.1368973884684922, '10.png'), (0.10035062431161372, '9.png'), (0.003744007402371596, '6.png'), (-0.016282192773904566, '7.png')]
[(4.931336213403033, '4.png'), (5.417565339664923, '5.png'), (14.206473893927633, '8.png'), (118.99473069111978, '6.png'), (124.85376912042297, '9.png'), (127.46875617934086, '3.png'), (171.0141086702477, '1.png'), (205.53732572274455, '2.png'), (230.05023244406289, '10.png'), (263.53594529481586, '7.png')]
[(0.7678928186151097, '4.png'), (0.7569464569105548, '5.png'), (0.4701646773610264, '8.png'), (0.3691336225019768, '9.png'), (0.36850067116029095, '1.png'), (0.3270427668094271, '3.png'), (0.31299965757534665, '10.png'), (0.30633111015777104, '2.png'), (0.1972729716799222, '6.png'), (0.1938923991547199, '7.png')]
[(0.3420150627382142, '5.png'), (0.3531722224349389, '4.p

In [12]:
#print (target_class_compare_dict['pikachu']['pikachu']["Correlation"])
#print (target_class_compare_dict['pikachu']['charmander']["Correlation"])
print (target_class_compare_dict['pikachu']['bulbasaur']["Chi-Squared"])
#print (target_class_compare_dict['pikachu']['squirtle']["Correlation"]) # now compare the targets and classes of different pokemon

[(1.5640142784473547, '8.png'), (10.053827906141446, '9.png'), (10.76300779186892, '6.png'), (23.969043773208583, '5.png'), (51.88595028699742, '2.png'), (62.66979879006354, '3.png'), (62.742862620225985, '7.png'), (84.25463774091708, '4.png'), (89.68251770080992, '1.png'), (231.1542099032823, '10.png')]


In [13]:
pTest = '0'
bulbTest = dict()
pikachuTest = dict()
charTest = dict()
squirtleTest = dict()
avg_score = 0
avg_score_dict = dict()
max_avg_score_dict = dict()
max_avg_score_dict_pikachu = dict()
max_avg_score_dict_squirtle = dict()
max_avg_score_dict_char = dict()
max_avg_score_dict_bulb = dict()
max_array = []
#preferredTest = dict()
p_test_pikachu = dict()
p_test_squirtle = dict()
p_test_bulb = dict()
p_test_char = dict()
def calculate_score_range(histogram_scores_tuple): #calculate the range of scores 
    preferredTest = dict()
    scores = [score for (score,filename) in histogram_scores_tuple] #grab the file scores from the target class compare results
    median_score = scores[int(len(scores)/2)]
    max_score = max(scores)
    min_score = min(scores)
    avg_score = sum(scores)/len(scores) #calculate the median avg max and min
    if(avg_score < median_score): 
        pTest= [methodName]
        preferredTest[methodName]= (avg_score)

    avg_score_dict[methodName] = avg_score

    return (avg_score, median_score, max_score, min_score, preferredTest, avg_score_dict)
def calc_avg_score(histogram_scores_tuple):
    theScores = [score for (score,filename) in histogram_scores_tuple] 
    avgScore = sum(theScores)/len(theScores)
    medianScore = theScores[int(len(theScores)/2)] #take the average score and the median score if the avgscore is less than the median that means we get a lot of strong hits
    if(avgScore < medianScore): 
        return (avgScore) #if it is less than then return the avg score else return 0
    else:
        return 0 
    
calc_score_dict = dict()
calc_correlation_dict = dict()
i = [0,1,2,3]
j = [0,1,2,3]
x = 3
pikachu_name = 'pikachu'
squirtle_name = 'squirtle'
bulb_name = 'bulbasaur'
char_name = 'charmander'

for (methodName, method, reverse) in OPENCV_METHODS:
    print (calculate_score_range(target_class_compare_dict[pikachu_name][pikachu_name][methodName])) #print the average max min and median
    max_avg_score_dict_pikachu[pikachu_name] = avg_score_dict # store the max average score for each test
    rand = calc_avg_score(target_class_compare_dict[pikachu_name][pikachu_name][methodName]) #store the average score only for the tests that have an avg less than the median
    if (rand != 0): #if it returns something other than zero then store the score with the test name
        pikachuTest[methodName] = rand
    print (calculate_score_range(target_class_compare_dict[squirtle_name][squirtle_name][methodName]))
    max_avg_score_dict_squirtle[squirtle_name] = avg_score_dict
    rand = calc_avg_score(target_class_compare_dict[squirtle_name][squirtle_name][methodName])
    if (rand != 0):
        squirtleTest[methodName] = rand
    print (calculate_score_range(target_class_compare_dict[bulb_name][bulb_name][methodName]))
    max_avg_score_dict_bulb[methodName] = avg_score_dict
    rando = calc_avg_score(target_class_compare_dict[bulb_name][bulb_name][methodName])
    if (rando != 0):
        bulbTest[methodName] = rando
    print (calculate_score_range(target_class_compare_dict[char_name][char_name][methodName]))
    max_avg_score_dict_char[char_name] = avg_score_dict
    rando = calc_avg_score(target_class_compare_dict[char_name][char_name][methodName])
    if (rando != 0):
        charTest[methodName] = rando

    print()


#print (max_avg_score_dict_pikachu)
#print (max_avg_score_dict_char)

print(bulbTest)
print(pikachuTest)
print(squirtleTest)
print(charTest['Correlation'])

(0.2965363064305562, 0.13955487527445715, 0.8578661145484695, -0.016282192773904566, {}, {'Correlation': 0.2965363064305562})
(0.1514228390502689, -0.13469173260029785, 0.927982643066907, -0.24768712263804848, {}, {'Correlation': 0.1514228390502689})
(0.12540807304591636, -0.05377971071306663, 0.7935632915327747, -0.3031291439284923, {}, {'Correlation': 0.12540807304591636})
(0.6040641630036514, 0.8762430517079068, 0.9648501528862824, 0.10541783508926617, {'Correlation': 0.6040641630036514}, {'Correlation': 0.6040641630036514})

(126.60102435697502, 127.46875617934086, 263.53594529481586, 4.931336213403033, {'Chi-Squared': 126.60102435697502}, {'Correlation': 0.6040641630036514, 'Chi-Squared': 126.60102435697502})
(9.578015977446471, 11.25632707365451, 23.144474813917498, 0.7415842370288771, {'Chi-Squared': 9.578015977446471}, {'Correlation': 0.6040641630036514, 'Chi-Squared': 9.578015977446471})
(9.364504374756896, 7.4134596193500935, 35.005377981356226, 0.770919824265436, {}, {'Corre

In [282]:
print(avg_score_dict)

{'Hellinger': 0.47130682561001114}


In [15]:
possible_squirtle_dict=dict()
possible_char_dict = dict()
possible_pikachu_dict = dict()
possible_bulb_dict = dict()

def greater_than_avg(the_method, myst_results_dict): # run the preferred test from above for each pokemon 
    greater_than_avg=dict()
    for (image_name_mystery, hist_base_two) in sorted(hist_mystery_dict.items()):
        for (img_name, target_score) in sorted(myst_results_dict[image_name_mystery][the_method].items()): 
            for (img_name_mystery, score) in (myst_results_dict[image_name_mystery][the_method].items()):
                if(score>average_score):
                    greater_than_avg[image_name_mystery] = score # return the mystery images that score above average for the preferred test
    return greater_than_avg

for (method_name, average_score) in (squirtleTest.items()):
    possible_squirtle_dict[method_name] = greater_than_avg(method_name, mystery_results_dict_squirtle) # run the greater than avg for all four pokemon and store the score 
print(possible_squirtle_dict)  #and image name as potentially a hit for that pokemon and do it for all four
print('Squirtle  ')

for (method_name, average_score) in (bulbTest.items()):
    possible_bulb_dict[method_name] = greater_than_avg(method_name, mystery_results_dict_bulb)
print(possible_bulb_dict)  
print('bulb ')
for (method_name, average_score) in (charTest.items()):
    possible_char_dict[method_name] = greater_than_avg(method_name, mystery_results_dict_char)
print(possible_char_dict)
print('char')

for (method_name, average_score) in (pikachuTest.items()):
    possible_pikachu_dict[method_name] = greater_than_avg(method_name, mystery_results_dict_pikachu)
print(possible_pikachu_dict)  

{'Chi-Squared': {'mystery1.png': 19.035500856377805, 'mystery2.png': 14.482979355922655, 'mystery3.png': 14.590161034958513, 'mystery6.png': 32.89657179115637, 'mystery8.png': 22.954080079992586}, 'Hellinger': {'mystery1.png': 0.7349577724675438, 'mystery2.png': 0.6240251331278139, 'mystery3.png': 0.6259042523949876, 'mystery6.png': 0.778065165643972, 'mystery8.png': 0.7039022386288077}}
Squirtle  
{'Hellinger': {'mystery1.png': 0.7487742013285382, 'mystery2.png': 0.6119421001712279, 'mystery3.png': 0.6246168394065275, 'mystery4.png': 0.5453211348696826, 'mystery6.png': 0.7964293967547684, 'mystery7.png': 0.5062754955097309, 'mystery8.png': 0.7281508806358205, 'mystery9.png': 0.5394314584613887}}
bulb 
{'Correlation': {'mystery10.png': 0.9082651810061503, 'mystery4.png': 0.8775633277413312, 'mystery5.png': 0.9338797740941831, 'mystery7.png': 0.8496687041254094, 'mystery9.png': 0.9211785909977198}, 'Intersection': {'mystery10.png': 0.7614872007397935, 'mystery4.png': 0.7163735327121685,

In [16]:

pSquirtleName = dict()
for (imag, sre) in (possible_squirtle_dict.items()):
    print(imag)
    print(sre)
    for(iname, so) in (sre.items()):
        print(iname)
        pSquirtleName[iname] = so
print(pSquirtleName) #just trying some things out for squirtle

Chi-Squared
{'mystery1.png': 19.035500856377805, 'mystery2.png': 14.482979355922655, 'mystery3.png': 14.590161034958513, 'mystery6.png': 32.89657179115637, 'mystery8.png': 22.954080079992586}
mystery1.png
mystery2.png
mystery3.png
mystery6.png
mystery8.png
Hellinger
{'mystery1.png': 0.7349577724675438, 'mystery2.png': 0.6240251331278139, 'mystery3.png': 0.6259042523949876, 'mystery6.png': 0.778065165643972, 'mystery8.png': 0.7039022386288077}
mystery1.png
mystery2.png
mystery3.png
mystery6.png
mystery8.png
{'mystery1.png': 0.7349577724675438, 'mystery2.png': 0.6240251331278139, 'mystery3.png': 0.6259042523949876, 'mystery6.png': 0.778065165643972, 'mystery8.png': 0.7039022386288077}


In [19]:
#match_dict = 0
def comparision_funct(target_name,not_likely_dict, mystery_results_dict, preferredTest,image_name, test, number):
    thresh = -.5
    avg = preferredTest[test] #last comaprision fucntion hopefully take the average from the test passed in

    if(image_name) in (not_likely_dict): #if the image is in the not likely dictionary then skip it in order to streamline the process
        return 0

    for (img_name, target_score) in sorted(mystery_results_dict[image_name][test].items()): #Now take the image name and target score from the mystery resluts dict
            #if(image_name_mystery in match_dict):
             #   continue
                #if(target_score > avg):
            pError = ((avg - target_score)/(target_score)) #using the good ole reliable percent error because if its negative it means its much higher than the avg so match
            if(pError < number): #if the p error is less than target number store the score and return the value to be put into a dict with the mystery image
                match_dict = target_score
                confirmedTest = image_name_mystery
                return(match_dict)#, confirmedTest) #so far is works better just returning the number but keeping it commeneted just in case
                    
            else:
                return 0 #if it isn't less than then return 0 to keep that area in the dictionary empty

def calc_avg_score_total(histogram_scores_tuple):
    theScores = [score for (score,filename) in histogram_scores_tuple]
    avgScore = sum(theScores)/len(theScores) #created a new function to return the average total of all the tests not just one
    return avgScore
test_number = -.84 #using .84 because online I gathered that .9 is a strong reliable number so taking that minus the average divided by should give a strong number thus .84
bulb_match = dict()
for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_bulb.items()): #now I'm starting with bulbasuar because it only has one test making it the easiest and 
    for (test) in bulbTest: #fastest to run to knock out some hits 
        compare = comparision_funct(bulb_name, not_likely_bulb_dict, mystery_results_dict_bulb, bulbTest, image_name_mystery, test, test_number) #call the compare function from above
        if compare != 0: #if it returns to us a value that isn't 0 meaning it is a match then store it in a dictionary
        #print(compare)
            bulb_match[image_name_mystery] = compare
print(bulb_match)      #print the match dictionary to see the matches    
if len(bulb_match) == 0: #if the length of the dictionary is 0 it means there are no matches so try again 
    for (methodName, method, reverse) in OPENCV_METHODS: #now this time since one test isn't conclusive we need to run all the test to see if we get any results
        max_avg_score_dict_bulb[methodName] = calc_avg_score_total(target_class_compare_dict[bulb_name][bulb_name][methodName]) #call this in order to get the average score of all the tests
        for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_bulb.items()):
            compare1 = comparision_funct(bulb_name, not_likely_bulb_dict, mystery_results_dict_bulb, max_avg_score_dict_bulb, image_name_mystery, methodName, test_number)
#recall the comparison function and run it for all the tests and try again
            if compare1 != 0:
                bulb_match[image_name_mystery] = compare1 #if the function returns something other than zeros it is a match
print(bulb_match) #hopefully it is a match
pikachu_match = dict()
pikachu_test_number=0 #now try the same method for pikachu we want matches
for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_pikachu.items()):
    if(image_name_mystery) in (bulb_match): #skip the images that are matches for bulbasaur to limit the number of images being passed
        print('hehe')
        continue
    for (test) in pikachuTest:
        compare = comparision_funct(pikachu_name, not_likely_pikachu_dict, mystery_results_dict_pikachu, pikachuTest, image_name_mystery, test, pikachu_test_number)
        if compare != 0:
            pikachu_match[image_name_mystery] = compare   
print(pikachu_match)
for (methodName, method, reverse) in OPENCV_METHODS: #same thing if it is empty run again
        max_avg_score_dict_pikachu[methodName] = calc_avg_score_total(target_class_compare_dict[pikachu_name][pikachu_name][methodName])
        for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_pikachu.items()):
            compare1 = comparision_funct(pikachu_name, not_likely_pikachu_dict, mystery_results_dict_pikachu, max_avg_score_dict_pikachu, image_name_mystery, methodName, test_number)
            if compare1 != 0:
                pikachu_match[image_name_mystery] = compare1    
print(pikachu_match)
print(bulb_match)

{}
{'mystery10.png': 0.9155755250086286, 'mystery5.png': 0.8310934660525131}
hehe
hehe
{'mystery1.png': 0.6877063363534573}
{'mystery1.png': 0.6877063363534573}
{'mystery10.png': 0.9155755250086286, 'mystery5.png': 0.8310934660525131}


In [20]:

test_number = 0
char_match = dict() #now run this all over again for charmander and skip the images in bulb match and pikachu match
char_char = dict()
for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_char.items()):
    
    if(image_name_mystery) in (bulb_match):
        continue
    if(image_name_mystery) in (pikachu_match):
        continue
    for (test) in charTest:
        compare = comparision_funct(char_name, not_likely_char_dict, mystery_results_dict_char, charTest, image_name_mystery, test, test_number)
        if compare != 0:
            char_match[image_name_mystery] = compare
            char_char[test] = char_match #messed around with creating a nested for loop just to see what would happen for the char match
print(char_match)
print(char_char)

{'mystery4.png': 0.7163735327121685, 'mystery7.png': 0.6924666644772515, 'mystery9.png': 0.7374292244303433}
{'Correlation': {'mystery4.png': 0.7163735327121685, 'mystery7.png': 0.6924666644772515, 'mystery9.png': 0.7374292244303433}, 'Intersection': {'mystery4.png': 0.7163735327121685, 'mystery7.png': 0.6924666644772515, 'mystery9.png': 0.7374292244303433}}


In [21]:
test_number = 0
squirtle_match = dict() #now run this all over again one last time for squirtle and skip the images in bulb match and pikachu match and char match
squirtle_squirtle = dict()
for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_squirtle.items()):
    
    if(image_name_mystery) in (bulb_match):
        continue
    if(image_name_mystery) in (pikachu_match):
        continue
    if(image_name_mystery) in (char_match):
        continue
    for (test) in squirtleTest:
        compare = comparision_funct(squirtle_name, not_likely_squirtle_dict, mystery_results_dict_squirtle, squirtleTest, image_name_mystery, test, test_number)
        if compare != 0:
            squirtle_match[image_name_mystery] = compare
            squirtle_squirtle[test] = char_match
print(squirtle_match)
print(squirtle_squirtle) #i acknowledge the fact that this is empty mainly because all the others are matches or not likely but for the sake of consistency and maybe this is ran with other images I wrote it for squirtle as well

{}
{}


In [22]:
no_match_dict = dict() #now go through the match dictionaries and find the images that didn't match with anything and store them in a no match dict
for (image_name_mystery, hist_base_two) in sorted(mystery_results_dict_squirtle.items()):
    if(image_name_mystery) in (bulb_match):
        continue
    if(image_name_mystery) in (pikachu_match):
        continue
    if(image_name_mystery) in (char_match):
        continue
    if(image_name_mystery) in (squirtle_match):
        continue
    no_match_dict[image_name_mystery] = image_name_mystery
print(no_match_dict)

print('The Bulbasaur matches are: ')
print(bulb_match)
print('The pikachu matches are: ')
print(pikachu_match)
print('The charmander matches are: ')
print(char_match)
print('The Squirtle matches are: ')
print(squirtle_match)
print('The no matches are :')
print(no_match_dict)

{'mystery2.png': 'mystery2.png', 'mystery3.png': 'mystery3.png', 'mystery6.png': 'mystery6.png', 'mystery8.png': 'mystery8.png'}
The Bulbasaur matches are: 
{'mystery10.png': 0.9155755250086286, 'mystery5.png': 0.8310934660525131}
The pikachu matches are: 
{'mystery1.png': 0.6877063363534573}
The charmander matches are: 
{'mystery4.png': 0.7163735327121685, 'mystery7.png': 0.6924666644772515, 'mystery9.png': 0.7374292244303433}
The Squirtle matches are: 
{}
The no matches are :
{'mystery2.png': 'mystery2.png', 'mystery3.png': 'mystery3.png', 'mystery6.png': 'mystery6.png', 'mystery8.png': 'mystery8.png'}
